## Task 2

In [25]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

**Primary keys for each table**
- Total population:   _Area Code_ & _Year Code_
- Animal products:    _Area Code_ & _Item Code_ & _Element_ & _Year Code_
- Vegetal products:   _Area Code_ & _Item Code_ & _Element_ & _Year Code_
- Cereals: _Item Code_
- People undernourished: _Area Code_ & _Year Code_

Or simply an auto-generated id.

After checking the **"Definition and standards"** section on the FAO website, we realised some elements are redundant.

We learned that:
      
      
    Production + Import Quantity - Export Quantity - Stock Variation = Domestic supply quantity  =  
    = Feed + Seed + Losses + Processing + Other uses (non-food) + Residuals + Food 

To illustrate this, we have taken the example of _Wheat and Products_ in France:


    38950 + 2394  -(-1209) - 22226 = 20327 = 7524 + 770 + 359 + 1788 + 2851 + 0 + 7035 
        
We conclude that the _Domestic supply quantity_ column is redundant. (Obviously, there are also a few descriptive columns like _Domain Code_ , _Domain_ , _Flag_ that are also considered redundant to our study)

In [26]:
wheat_france = pd.read_csv("../data/latest_v3/veg_prod_2014.csv")
wheat_france = wheat_france[(wheat_france['Area']=='France') & (wheat_france['Item'] == 'Wheat and products')].reset_index()
wheat_france[['Area', 'Element', 'Item', 'Value']][:12]

,Area,Element,Item,Value
0,France,Production,Wheat and products,38950.0
1,France,Import Quantity,Wheat and products,2394.0
2,France,Stock Variation,Wheat and products,-1209.0
3,France,Export Quantity,Wheat and products,22226.0
4,France,Domestic supply quantity,Wheat and products,20327.0
5,France,Feed,Wheat and products,7524.0
6,France,Seed,Wheat and products,770.0
7,France,Losses,Wheat and products,359.0
8,France,Processing,Wheat and products,1788.0
9,France,Other uses (non-food),Wheat and products,2851.0


In [27]:
# Creating our initial population dataframe
population_data = pd.read_csv("../data/latest_v3/total_population.csv")
population_df = pd.DataFrame(data = population_data)
population_df.head()

,Domain Code,Domain,Area Code,Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,FBS,New Food Balances,2,Afghanistan,511,Total Population - Both sexes,2501,Population,2014,2014,1000 persons,33370.79,S,Standardized data
1,FBS,New Food Balances,2,Afghanistan,511,Total Population - Both sexes,2501,Population,2017,2017,1000 persons,36296.11,S,Standardized data
2,FBS,New Food Balances,3,Albania,511,Total Population - Both sexes,2501,Population,2014,2014,1000 persons,2896.30,S,Standardized data
3,FBS,New Food Balances,3,Albania,511,Total Population - Both sexes,2501,Population,2017,2017,1000 persons,2884.17,S,Standardized data
4,FBS,New Food Balances,4,Algeria,511,Total Population - Both sexes,2501,Population,2014,2014,1000 persons,38923.69,S,Standardized data


Calculating the world population for 2014 and 2017

In [28]:
# Creating a dataframe of population that contains both 2014 and 2017 population data
# Keeping only the entries that are of the form "Standardized data" so as to eliminate duplicate information
# e.g. China has both 2 sets of entries: standardized (per regions) and aggregated
pop_2014_and_2017 = population_df[population_df['Flag Description'] == 'Standardized data']

# Grouping by year
pop_2014_and_2017 = pop_2014_and_2017.groupby(['Year'])

# Creating an array containing two elememnts, one for 2014 and one for 2017
pop_2014_and_2017_array = np.array(pop_2014_and_2017)

# Saving the sum over the population data for 2014 and 2017 respectively
world_population_2014 = int(1000*pop_2014_and_2017_array[0][1][['Value']].sum())
world_population_2017 = int(1000*pop_2014_and_2017_array[1][1][['Value']].sum())

# Creating a list with two entries, each entry showing the year and the world population calculated for that year
world_population = [[2014, world_population_2014], [2017,world_population_2017]]
pd.DataFrame(np.array(world_population), columns = ['year', 'world_population'])

,year,world_population
0,2014,7089471330
1,2017,7329868959


## Task 3

In [29]:
#FAO Data - Cleaning - Paul
# The goal is to format the main dataframe (food balance) to have a column for each element, e.g.
# Production, Importation, Domestic supply, Seed, etc.,
# This way, the data manipulation will be much easier!


# Loads the food balance data
veg1 = pd.read_csv("../data/latest_v3/veg_prod_2014.csv")
veg2 = pd.read_csv("../data/latest_v3/veg_prod_2017.csv")
ani = pd.read_csv("../data/latest_v3/animal_products.csv")
veg = veg1.append(veg2)

# Adds variable ‘origin’
ani["origin"] = "animal"
veg["origin"] = "vegetal"

# Appends veg and ani to one dataframe
temp = ani.append(veg)

#Deletion of ani et veg
del ani, veg

#Renaming of temp’s columns
temp.columns = ["xx", "xx2", "country_code", "country", 'xx3', 'element'
,'item_code', 'item', 'xx4', "year", "unit", "value", 'xx5', 'xx6'
, 'origin']

# Transformation of ‘temp’ to a pivot table
data = temp.pivot_table(
index=["country_code", "country", "item_code", "item", "year", "origin"],
columns = ["element"], values=["value"], aggfunc=sum)


# Renaming of data’s columns (be careful, the order of the columns needs
# to be adapted to your own data !)
# Yes, I was, I had to add 'residuals', 'tourist_consumption' and 'fat_supply_quantity_gcapitaday'
data.columns = ['domestic_supply_quantity', 'export_quantity', 'fat_supply_quantity_gcapitaday', 'feed',
'food', 'food_supply_kcalcapitaday',
'food_supply_quantity_kgcapitayr', 'import_quantity', 'losses', 'other_uses',
'processing', 'production', 'protein_supply_quantity_gcapitaday',
'residuals', 'seed', 'stock_variation', 'tourist_consumption']

# Getting rid of the three extra columns post pivotation
data = data[['domestic_supply_quantity', 'export_quantity', 'feed',
'food', 'food_supply_kcalcapitaday',
'food_supply_quantity_kgcapitayr', 'import_quantity', 'losses', 'other_uses',
'processing', 'production', 'protein_supply_quantity_gcapitaday',
 'seed', 'stock_variation']]


#Index columns need to be normal columns
data = data.reset_index()
data = data[data.country_code != 351]
data.sample()

,country_code,country,item_code,item,year,origin,domestic_supply_quantity,export_quantity,feed,food,food_supply_kcalcapitaday,food_supply_quantity_kgcapitayr,import_quantity,losses,other_uses,processing,production,protein_supply_quantity_gcapitaday,seed,stock_variation
9162,74,Gabon,2680,Infant food,2014,vegetal,2.0,0.0,NaN,2.0,11.0,1.11,2.0,NaN,NaN,NaN,NaN,0.46,NaN,0.0


## Task 4

To carry out your future study, you’ll need to compute new variables from the one you already have.

Here is the list of variables:

- **_food_supply_kcal_ and _food_supply_kgprotein_**


In [30]:
# Creating a lite version of our population_df dataframe
pop_data = population_df[['Area Code', 'Area', 'Year', 'Value']]
pop_data['Value'] = pop_data['Value']*1000
pop_data.columns = ['country_code', 'country', 'year', 'population']

# Merging the population dataframe with the global food balance sheet dataframe
data1 = pd.merge(data, pop_data, on=['country_code', 'country', 'year'] )

# Creating the food_supply_kcal and food_supply_kgprotein columns
data1['food_supply_kcal'] = data1['food_supply_kcalcapitaday']*365*data1['population']
data1['food_supply_kgprotein'] = (data1['protein_supply_quantity_gcapitaday']*data1['population']*365)/1000

data1[['country_code','country','item_code','item','year','origin', 'food_supply_kcal', 'food_supply_kgprotein']].sample()

,country_code,country,item_code,item,year,origin,food_supply_kcal,food_supply_kgprotein
21945,170,Peru,2766,Cephalopods,2014,animal,5.491491e+10,1.120264e+07


- **_food_supply_kg_**

In [31]:
# Creating the food_supply_kg column
data1['food_supply_kg'] = data1['food']*(10**6)

data1[['country_code','country','item_code','item','year','origin', 'food_supply_kg']].head()

,country_code,country,item_code,item,year,origin,food_supply_kg
0,1,Armenia,2511,Wheat and products,2014,vegetal,380000000.0
1,1,Armenia,2513,Barley and products,2014,vegetal,3000000.0
2,1,Armenia,2514,Maize and products,2014,vegetal,18000000.0
3,1,Armenia,2515,Rye and products,2014,vegetal,0.0
4,1,Armenia,2516,Oats,2014,vegetal,1000000.0


- **_ratio_kcalkg_ and _protein_percentage_**

In [32]:
# Creating the ratio_kcalkg and protein_percentage columns
# We need a mask in order to avoid division by 0
mask = data1["food_supply_kg"] != 0 
data1.loc[mask, 'ratio_kcalkg'] = data1.loc[mask, "food_supply_kcal"] / data1.loc[mask, "food_supply_kg"]
data1.loc[mask, 'protein_percentage'] = 100*data1.loc[mask, "food_supply_kgprotein"] / data1.loc[mask, "food_supply_kg"]

data1[['country_code','country','item_code','item','year','origin', 'ratio_kcalkg', 'protein_percentage']].head(10)

,country_code,country,item_code,item,year,origin,ratio_kcalkg,protein_percentage
0,1,Armenia,2511,Wheat and products,2014,vegetal,2906.546859,8.666489
1,1,Armenia,2513,Barley and products,2014,vegetal,2834.745733,9.567267
2,1,Armenia,2514,Maize and products,2014,vegetal,3543.432167,18.543962
3,1,Armenia,2515,Rye and products,2014,vegetal,NaN,NaN
4,1,Armenia,2516,Oats,2014,vegetal,5315.148250,20.197563
5,1,Armenia,2517,Millet and products,2014,vegetal,NaN,NaN
6,1,Armenia,2518,Sorghum and products,2014,vegetal,NaN,NaN
7,1,Armenia,2520,"Cereals, Other",2014,vegetal,3189.088950,8.656099
8,1,Armenia,2531,Potatoes and products,2014,vegetal,685.258783,1.598937
9,1,Armenia,2532,Cassava and products,2014,vegetal,NaN,NaN


- **_dom_sup_kcal_ and _dom_sup_kgprot_**

In [33]:
# Creating the dom_sup_kcal and dom_sup_kgprot columns
data1['dom_sup_kcal'] = data1['domestic_supply_quantity']*(10**6)*data1['ratio_kcalkg']
data1['dom_sup_kgprot'] = (data1['domestic_supply_quantity']*(10**6)*data1['protein_percentage'])/100

data1[['country_code','country','item_code','item','year','origin', 'dom_sup_kcal', 'dom_sup_kcal']]

,country_code,country,item_code,item,year,origin,dom_sup_kcal,dom_sup_kcal
0,1,Armenia,2511,Wheat and products,2014,vegetal,1.857283e+12,1.857283e+12
1,1,Armenia,2513,Barley and products,2014,vegetal,5.556102e+11,5.556102e+11
2,1,Armenia,2514,Maize and products,2014,vegetal,3.118220e+11,3.118220e+11
3,1,Armenia,2515,Rye and products,2014,vegetal,NaN,NaN
4,1,Armenia,2516,Oats,2014,vegetal,4.252119e+10,4.252119e+10
...,...,...,...,...,...,...,...,...
31728,276,Sudan,2781,"Fish, Body Oil",2017,animal,NaN,NaN
31729,276,Sudan,2782,"Fish, Liver Oil",2017,animal,NaN,NaN
31730,276,Sudan,2805,Rice and products,2017,vegetal,3.328524e+11,3.328524e+11
31731,276,Sudan,2848,Milk - Excluding Butter,2017,animal,3.573661e+12,3.573661e+12


- **_great_import_from_undern_countries_**

In [34]:
# First, I read the people_undernourished.csv file and create our people_undernourished_df dataframe
people_undernourished_df = pd.read_csv("../data/latest_v3/people_undernourished.csv")

# Formatting the year data, from 201(x-1) - 201(x+1) to 201x
dictionary_year = {'2012-2014': 2013, '2013-2015':2014, '2014-2016': 2015, '2015-2017':2016, '2016-2018': 2017}
people_undernourished_df['Year'] = people_undernourished_df['Year'].map(dictionary_year)

# Creating another dataframe, this time keeping only the 'Area Code', 'Area', 'Year', 'Value' columns
undernourished_data = people_undernourished_df[['Area Code', 'Area', 'Year', 'Value']]

# Renaming the columns to the names mentioned in Task 3
undernourished_data.columns = ['country_code', 'country', 'year', 'u_population']

# Keeping only the data for 2014 and 2017
undernourished_data = undernourished_data[(undernourished_data.year != 2013) & (undernourished_data.year != 2015) & (undernourished_data.year != 2016)]

# Formatting the data for the 'u_population' column; 
# also, eliminating the rows coresponding to 'China' as they present aggregated data
undernourished_data['u_population'] = pd.to_numeric(undernourished_data['u_population'], errors ='coerce')
undernourished_data['u_population'] *= 1000000
undernourished_data = undernourished_data[undernourished_data.country_code != 351]

# Here we have a striped version of our initial population_df dataframe containing only the 'Area Code', 'Area', 'Year', 'Value' columns
# Renaming the columns accordingly for these too
# Formatting the 'population' column accordingly
# Eliminating the rows for 'China' due to aggregated data
pop_data = population_df[['Area Code', 'Area', 'Year', 'Value']]
pop_data.columns = ['country_code', 'country', 'year', 'population']
pop_data['population'] = pop_data['population']*1000
pop_data = pop_data[pop_data.country_code != 351]

# Merging the undernourished_data and the pop_data dataframes
pop_data2 = pd.merge(undernourished_data, pop_data, on=['country_code', 'country', 'year'] )
mask = pop_data2["population"] != 0 # because it's not possible to divide by 0

# Finally calculating the percentage of undernourished people for each country for our pop_data2 dataframe
pop_data2.loc[mask, 'u_percentage'] = 100*pop_data2.loc[mask, "u_population"]/pop_data2.loc[mask, "population"]
pop_data2 = pop_data2[['country_code', 'country', 'year', 'population', 'u_population','u_percentage']]
pop_data2.head()


,country_code,country,year,population,u_population,u_percentage
0,2,Afghanistan,2014,33370790.0,8800000.0,26.370368
1,2,Afghanistan,2017,36296110.0,10600000.0,29.204232
2,3,Albania,2014,2896300.0,200000.0,6.905362
3,3,Albania,2017,2884170.0,200000.0,6.934404
4,4,Algeria,2014,38923690.0,1600000.0,4.110607


In [35]:
# Creating a merged dataframe containing the initial global food balance sheet data, 
# including the extra 7 columns created during Task 4 so far
data2 = pd.merge(data1, pop_data2, on=['country_code', 'country', 'year'] )
data2 = data2.rename(columns={'population_y':'population'})

data2.head()

,country_code,country,item_code,item,year,origin,domestic_supply_quantity,export_quantity,feed,food,...,food_supply_kcal,food_supply_kgprotein,food_supply_kg,ratio_kcalkg,protein_percentage,dom_sup_kcal,dom_sup_kgprot,population,u_population,u_percentage
0,1,Armenia,2511,Wheat and products,2014,vegetal,639.0,1.0,110.0,380.0,...,1.104488e+12,3.293266e+07,380000000.0,2906.546859,8.666489,1.857283e+12,5.537887e+07,2912410.0,100000.0,3.433582
1,1,Armenia,2513,Barley and products,2014,vegetal,196.0,0.0,145.0,3.0,...,8.504237e+09,2.870180e+05,3000000.0,2834.745733,9.567267,5.556102e+11,1.875184e+07,2912410.0,100000.0,3.433582
2,1,Armenia,2514,Maize and products,2014,vegetal,88.0,NaN,64.0,18.0,...,6.378178e+10,3.337913e+06,18000000.0,3543.432167,18.543962,3.118220e+11,1.631869e+07,2912410.0,100000.0,3.433582
3,1,Armenia,2515,Rye and products,2014,vegetal,1.0,NaN,1.0,0.0,...,1.063030e+09,3.189089e+04,0.0,NaN,NaN,NaN,NaN,2912410.0,100000.0,3.433582
4,1,Armenia,2516,Oats,2014,vegetal,8.0,NaN,6.0,1.0,...,5.315148e+09,2.019756e+05,1000000.0,5315.148250,20.197563,4.252119e+10,1.615805e+06,2912410.0,100000.0,3.433582


    - top 25 exports

In [36]:
# Now, creating the two dataframes containing the 25 most exported items from countries with an undernourished perc. over 10%
# for 2014, and 2017 respectively
top_25_exports = data2[data2.u_percentage >=10]
top_25_exports_2014 = top_25_exports[top_25_exports.year == 2014].groupby([ 'year','item_code','item'], as_index = False)[['export_quantity']].sum().sort_values(by=['export_quantity'], ascending=False).head(25)
top_25_exports_2017 = top_25_exports[top_25_exports.year == 2017].groupby([ 'year','item_code','item'], as_index = False)[['export_quantity']].sum().sort_values(by=['export_quantity'], ascending=False).head(25)

# Concatenating the above dataframes into a single one
top_25_exports = pd.concat([top_25_exports_2014, top_25_exports_2017])
top_25_exports = top_25_exports.reset_index(drop = True)

top_25_exports.sample()

,year,item_code,item,export_quantity
9,2014,2625,"Fruits, Other",2964.0


    - top 200 imports

In [37]:
# Then, here we have the lists with the 25 most exported items in 2014 and 2017, respectively
most_exported_items_2014 = list(top_25_exports_2014['item'])
most_exported_items_2017 = list(top_25_exports_2017['item'])

# The dataframe containing the top 200 imports (of the 25 most exported items) in 2014
top_200_imports_2014 = data2[data2.year==2014]
top_200_imports_2014 = top_200_imports_2014[top_200_imports_2014.item.isin(most_exported_items_2014)].groupby(['country', 'item', 'year'], as_index = False)[['import_quantity']].sum().sort_values(by=['import_quantity'], ascending=False).head(200)

# The dataframe containing the top 200 imports (of the 25 most exported items) in 2017
top_200_imports_2017 = data2[data2.year==2017]
top_200_imports_2017 = top_200_imports_2017[top_200_imports_2017.item.isin(most_exported_items_2017)].groupby(['country', 'item', 'year'], as_index = False)[['import_quantity']].sum().sort_values(by=['import_quantity'], ascending=False).head(200)

# The concatenated dataframe of the top 200 imports of both 2014 and 2017
# No. of rows = 400
top_200_imports = pd.concat([top_200_imports_2014, top_200_imports_2017])
top_200_imports = top_200_imports.reset_index(drop = True)

# Creating the new column 'great_import_from_undern_countries' and setting the values for all entries to True
top_200_imports['great_import_from_undern_countries'] = True

top_200_imports.sample()

,country,item,year,import_quantity,great_import_from_undern_countries
288,"China, mainland",Sugar (Raw Equivalent),2017,2381.0,True


**Finally, our global food balance sheet data containing the extra 8 columns created during Task 4**

In [38]:
# Creating our final dataframe containing all 32 columns (including the 8 new created during Task 4)
# through a merging process between our data2 and top_200_imports dataframes
data3 = pd.merge(data2, top_200_imports, on=['country','item','year','import_quantity'], how='left' )

# Setting the NaN values to False (so we only have 400 values - 200 for each year)
data3["great_import_from_undern_countries"] = data3["great_import_from_undern_countries"].fillna(False)
data3.head()

,country_code,country,item_code,item,year,origin,domestic_supply_quantity,export_quantity,feed,food,...,food_supply_kgprotein,food_supply_kg,ratio_kcalkg,protein_percentage,dom_sup_kcal,dom_sup_kgprot,population,u_population,u_percentage,great_import_from_undern_countries
0,1,Armenia,2511,Wheat and products,2014,vegetal,639.0,1.0,110.0,380.0,...,3.293266e+07,380000000.0,2906.546859,8.666489,1.857283e+12,5.537887e+07,2912410.0,100000.0,3.433582,False
1,1,Armenia,2513,Barley and products,2014,vegetal,196.0,0.0,145.0,3.0,...,2.870180e+05,3000000.0,2834.745733,9.567267,5.556102e+11,1.875184e+07,2912410.0,100000.0,3.433582,False
2,1,Armenia,2514,Maize and products,2014,vegetal,88.0,NaN,64.0,18.0,...,3.337913e+06,18000000.0,3543.432167,18.543962,3.118220e+11,1.631869e+07,2912410.0,100000.0,3.433582,False
3,1,Armenia,2515,Rye and products,2014,vegetal,1.0,NaN,1.0,0.0,...,3.189089e+04,0.0,NaN,NaN,NaN,NaN,2912410.0,100000.0,3.433582,False
4,1,Armenia,2516,Oats,2014,vegetal,8.0,NaN,6.0,1.0,...,2.019756e+05,1000000.0,5315.148250,20.197563,4.252119e+10,1.615805e+06,2912410.0,100000.0,3.433582,False


## Task 5

**1. Considering only plant products, what proportion of the global domestic supply is used as food, feed, losses and other uses.**

In [39]:
# Extracting a list with the sum values over the 'food', 'feed', 'losses', 'other_uses' columns
q1a= data1[data1.origin=='vegetal'].groupby(['year'], as_index = False)[['food', 'feed', 'losses', 'other_uses']].sum()

# Getting our float value of the 'domestic_supply_quantity' column
q1b = data1[data1.origin=='vegetal'].groupby(['year'], as_index = False)[['domestic_supply_quantity']].sum()

# Dividing the values in our q1a list by the 'domestic_supply_quantity' column value to get our list of proportions
q1a['domestic_supply_quantity'] = q1b['domestic_supply_quantity']
q1a=  pd.melt(q1a, id_vars=['year', 'domestic_supply_quantity'])
q1a['proportion'] = (100*q1a['value']/ q1a['domestic_supply_quantity']).round(2)
q1a = q1a[['year','variable', 'proportion']]
q1a = q1a.groupby(['year', 'variable']).sum()
q1a

proportion
year variable              
2014 feed             14.22
     food             42.29
     losses            6.06
     other_uses        8.78
2017 feed             14.48
     food             41.89
     losses            6.03
     other_uses        9.06

**2. How many humans on earth could be fed if all the plant-based food supply (crops), including food and feed, was used for human consumption? Give the results in terms of calories, and protein. Express these two results as a percentage of the world's population.**



In [40]:
# Creating our dataframe, by filtering the big dataframe (data2) by the "origin == 'vegetal'" condition
# while keeping only a handful of columns
q2a= data2[data2.origin=='vegetal']
q2a = q2a[['country','year', 'item_code', 'item', 'feed', 'food', 'ratio_kcalkg', 'protein_percentage']]

# Creating the two new columns kcal and food_feed_kg_protein
q2a['kcal'] = (q2a['feed'] + q2a['food']) * 1000000* q2a['ratio_kcalkg']
q2a['food_feed_kg_protein'] = (1000000*(q2a['feed'] + q2a['food'])) * q2a['protein_percentage']/100

# Replacing the NaN values with zeros
q2a["kcal"] = q2a["kcal"].fillna(0)
q2a["food_feed_kg_protein"] = q2a["food_feed_kg_protein"].fillna(0)

# Grouping by year and summing the two new columns
q2a = q2a.groupby(['year'], as_index = False)[['kcal', 'food_feed_kg_protein']].sum()

# Daily calorie and protein requirements for the average human
daily_calorie_req = 2500 # [kcal]
avg_body_weight = 62 # [kg]
dri_protein = 0.008 # daily recommended intake; [g/kg] of body weight

# Creating new columns calculating the amount of people that could've been fed (in 2014 and 2017)
# First, by the amount of kcal in the crops
# Second, by the amount of protein in the crops
q2a['people_fed_bykcal']=q2a['kcal']/daily_calorie_req/365
q2a['people_fed_bykcal'] = q2a['people_fed_bykcal'].round(0)

q2a['people_fed_bykgprotein'] = q2a['food_feed_kg_protein']/(avg_body_weight*dri_protein)/365
q2a['people_fed_bykgprotein'] = q2a['people_fed_bykgprotein'].round(0)

# Creating the population dataframe and calculating the total population for both 2014 and 2017
q2b = pd.read_csv("../data/latest_v3/total_population.csv")
q2b = q2b[q2b['Flag Description'] == 'Standardized data']
q2b = 1000*q2b.groupby(['Year'])[['Value']].sum()

# Merging our two dataframes and finding our final values
q2c = pd.merge(q2a, q2b, left_on='year', right_on='Year')
q2c = q2c.rename(columns={"Value": "Population"})
q2c['percentage_w_pop_fed_bykcal'] = 100*q2c['people_fed_bykcal']/q2c['Population']
q2c['percentage_w_pop_fed_bykgprotein'] = 100*q2c['people_fed_bykgprotein']/q2c['Population']
q2c[['year','percentage_w_pop_fed_bykcal', 'percentage_w_pop_fed_bykgprotein']]


,year,percentage_w_pop_fed_bykcal,percentage_w_pop_fed_bykgprotein
0,2014,125.175239,14.840902
1,2017,112.624625,14.604608


**3. How many humans could be fed with the global food supply? Give the results in terms of calories and protein. Express these two results as a percentage of the world's population.**


In [41]:
# Creating our dataframe, this time keeping the big dataframe (data2) as it is
# while still keeping only a handful of columns
q3a= data2
q3a = q3a[['country','year', 'item_code', 'item', 'feed', 'food', 'ratio_kcalkg', 'protein_percentage']]

# Creating the two new columns kcal and food_feed_kg_protein
q3a['kcal'] = (q3a['feed'] + q3a['food']) * 1000000 * q3a['ratio_kcalkg']
q3a['food_feed_kg_protein'] = 1000000*(q3a['feed'] + q3a['food']) * (q3a['protein_percentage']/100)

# Replacing the NaN values with zeros
q3a["kcal"] = q3a["kcal"].fillna(0)
q3a["food_feed_kg_protein"] = q3a["food_feed_kg_protein"].fillna(0)

# Grouping by year and summing the two new columns
q3a = q3a.groupby(['year'], as_index = False)[['kcal', 'food_feed_kg_protein']].sum()

# Daily calorie and protein requirements for the average human
daily_calorie_req = 2500 # [kcal]
avg_body_weight = 62 # [kg]
dri_protein = 0.008 # daily recommended intake; [g/kg] of body weight

# Creating new columns calculating the amount of people that could've been fed (in 2014 and 2017)
# First, by the amount of kcal in the crops
q3a['people_fed_bykcal']=q3a['kcal']/daily_calorie_req/365
q3a['people_fed_bykcal'] = q3a['people_fed_bykcal'].round(0)

# Second, by the amount of protein in the crops
q3a['people_fed_bykgprotein'] = q3a['food_feed_kg_protein']/(avg_body_weight*dri_protein)/365
q3a['people_fed_bykgprotein'] = q3a['people_fed_bykgprotein'].round(0)

# Creating the population dataframe and calculating the total population for both 2014 and 2017
q3b = pd.read_csv("../data/latest_v3/total_population.csv")
q3b = q3b[q3b['Flag Description'] == 'Standardized data']
q3b = 1000*q3b.groupby(['Year'])[['Value']].sum()

# Merging our two dataframes and finding our final values
q3c = pd.merge(q3a, q3b, left_on='year', right_on='Year')
q3c = q3c.rename(columns={"Value": "Population"})
q3c['percentage_w_pop_fed_bykcal'] = 100*q3c['people_fed_bykcal']/q3c['Population']

q3c['percentage_w_pop_fed_bykgprotein'] = 100*q3c['people_fed_bykgprotein']/q3c['Population']

q3c[['year','percentage_w_pop_fed_bykcal', 'percentage_w_pop_fed_bykgprotein']]


,year,percentage_w_pop_fed_bykcal,percentage_w_pop_fed_bykgprotein
0,2014,140.873571,19.329163
1,2017,122.114983,17.819073


**4. From the collected data on undernutrition, what proportion of the world's population is considered undernourished?**

In [42]:
# Creating a dataframe from the previously created population dataframe, 
# keeping and summing only the 'population', 'u_population' columns
# after grouping by 'year'
q4a = pop_data2.groupby(['year'])[['population', 'u_population']].sum()

# Creating a list of the two values of undernourished proportion in 2014 and 2017 respectively
q4b = 100*q4a.u_population/q4a.population

# Creating our mini-dataframe using our q4b list
q4c= pd.DataFrame({'proportion':q4b}, [2014, 2017])

q4c

,proportion
2014,10.325170
2017,10.202092


**5. Considering the 25 items most exported by the countries with a high rate of undernutrition, which three of them:**

- Have the greatest _other_uses_ to _domestic_supply_quantity_ ratio and what are they used for?

In [43]:
# Creating a dataframe that only includes the 25 most exported items in 2014, 
# while keeping only the 'country', 'item', 'year','other_uses', 'domestic_supply_quantity' columns
q5a_2014 = data2[data2.year==2014]
q5a_2014 = q5a_2014[q5a_2014.item.isin(most_exported_items_2014)].groupby(['country', 'item', 'year'], as_index = False)[['other_uses', 'domestic_supply_quantity']].sum()

# Creating our 'other_uses' to 'domestic_supply_quantity' ratio column
maskq5a = q5a_2014["domestic_supply_quantity"] != 0  
q5a_2014.loc[maskq5a,'ratio'] = q5a_2014.loc[maskq5a, 'other_uses']/q5a_2014.loc[maskq5a, 'domestic_supply_quantity']
q5a_2014.ratio = q5a_2014.ratio.fillna(0)
q5a_2014 = q5a_2014.sort_values(by=['ratio'], ascending = False).head(3)

# Ditto for 2017
q5a_2017 = data2[data2.year==2017]
q5a_2017 = q5a_2017[q5a_2017.item.isin(most_exported_items_2017)].groupby(['country', 'item', 'year'], as_index = False)[['other_uses', 'domestic_supply_quantity']].sum()
maskq5a = q5a_2017["domestic_supply_quantity"] != 0  
q5a_2017.loc[maskq5a,'ratio'] = q5a_2017.loc[maskq5a, 'other_uses']/q5a_2017.loc[maskq5a, 'domestic_supply_quantity']
q5a_2017.ratio = q5a_2017.ratio.fillna(0)
q5a_2017 = q5a_2017.sort_values(by=['ratio'], ascending = False).head(3)

# Concatenating the two dataframes 
q5a = pd.concat([q5a_2014, q5a_2017]).reset_index(drop=True)

q5a 

,country,item,year,other_uses,domestic_supply_quantity,ratio
0,Egypt,Cassava and products,2014,2.0,1.0,2.000000
1,"China, Taiwan Province of",Cassava and products,2014,337.0,308.0,1.094156
2,Italy,Cassava and products,2014,2.0,2.0,1.000000
3,Spain,"Sweeteners, Other",2017,123.0,28.0,4.392857
4,Malaysia,"Sweeteners, Other",2017,58.0,22.0,2.636364
5,United Arab Emirates,Cocoa Beans and products,2017,5.0,2.0,2.500000


- Have the greatest _feed_ to _(food+feed)_ ratio and what are they used for?

In [44]:
# Creating a dataframe that only includes the 25 most exported items in 2014, 
# while keeping only the 'country', 'item', 'year','other_uses', 'domestic_supply_quantity' columns
q5b_2014 = data2[data2.year==2014]
q5b_2014 = q5b_2014[q5b_2014.item.isin(most_exported_items_2014)].groupby(['country', 'item', 'year'], as_index = False)[['food', 'feed']].sum()

# Creating our 'other_uses' to 'domestic_supply_quantity' ratio column
maskq5b = (q5b_2014['food'] + q5b_2014['feed']) != 0  
q5b_2014.loc[maskq5b,'ratio'] = q5b_2014.loc[maskq5b, 'feed']/(q5b_2014.loc[maskq5b, 'feed'] + q5b_2014.loc[maskq5b, 'food'])
q5b_2014.ratio = q5b_2014.ratio.fillna(0)
q5b_2014 = q5b_2014.sort_values(by=['ratio'], ascending = False).head(3)

# Ditto for 2017
q5b_2017 = data2[data2.year==2017]
q5b_2017 = q5b_2017[q5b_2017.item.isin(most_exported_items_2017)].groupby(['country', 'item', 'year'], as_index = False)[['food', 'feed']].sum()
maskq5b = (q5b_2017['food'] + q5b_2017['feed']) != 0  
q5b_2017.loc[maskq5b,'ratio'] = q5b_2017.loc[maskq5b, 'feed']/(q5b_2017.loc[maskq5b, 'feed']+q5b_2017.loc[maskq5b, 'food'])
q5b_2017.ratio = q5b_2017.ratio.fillna(0)
q5b_2017 = q5b_2017.sort_values(by=['ratio'], ascending = False).head(3)

# Concatenating the two dataframes 
q5b = pd.concat([q5b_2014, q5b_2017]).reset_index(drop=True)

q5b 

,country,item,year,food,feed,ratio
0,Italy,Cassava and products,2014,0.0,1.0,1.0
1,Venezuela (Bolivarian Republic of),Soyabeans,2014,0.0,54.0,1.0
2,Tajikistan,Soyabeans,2014,0.0,2.0,1.0
3,Benin,Soyabeans,2017,0.0,2.0,1.0
4,Sri Lanka,Soyabeans,2017,0.0,1.0,1.0
5,Turkey,Cassava and products,2017,0.0,259.0,1.0


**6. Taking only grains (cereals) for food and feed into account, what proportion (in terms of weight) is used for feed?**

In [45]:
# Creating our list of cereal items
cereals_data = pd.read_csv("../data/latest_v3/cereals.csv")
cereals = cereals_data['Item'].unique()
cereals

# Creating our dataframe that only includes the cereals items; 
# Grouped by year and item
# Kept and summed the food and feed columns
q6 = data2[data2['item'].isin(cereals)].groupby(['year'], as_index = False)[['food', 'feed']].sum()

# Creating a mask to ensure no Inf values appear (due to division by 0)
mask6 = (q6["food"]+ q6["feed"]) != 0 

# Creating our final results (i.e. proportion column)
q6.loc[mask6,'percentage'] = round(100*q6.loc[mask6, 'feed']/(q6.loc[mask6, 'food'] + q6.loc[mask6, 'feed']))
q6[['year', 'percentage']]


,year,percentage
0,2014,43.0
1,2017,44.0


**7. How many tons of grains (cereals) could be released if the US reduced its production of animal products by 10%? Convert this quantity to kcal, and the number of potentially fed humans.**

In [46]:
# Creating a dataframe containing only the cereals entries for the 'United States of America' for 2014
q7 = data2[data2.item.isin(cereals)]
q7 = q7[q7.country=='United States of America']
q7 = q7[q7.year == 2014]

# 10% of (cereals) feed in kg
q7a = q7.feed.sum()*1000*0.1 

# 10% of (cereals) feed in kcal
q7b = round((q7.feed*1000*0.1  * q7.ratio_kcalkg).sum(), 0)

# no. of potentially fed humans with the 10% cereals feed
q7c = round(q7b/daily_calorie_req, 0)

q7d = pd.DataFrame(np.array([[q7a, q7b, q7c]]), columns=['10%_feed_kg', '10%_feed_kcal', 'potential_humans_fed'])
q7d

,10%_feed_kg,10%_feed_kcal,potential_humans_fed
0,14256200.0,3.944932e+10,15779727.0


In [47]:
# Creating a dataframe containing only the cereals entries for the 'United States of America' for 2017
q71 = data2[data2.item.isin(cereals)]
q71 = q71[q71.country=='United States of America']
q71 = q71[q71.year == 2017]

# 10% of (cereals) feed in kg
q71a = q71.feed.sum()*1000*0.1 

# 10% of (cereals) feed in kcal
q71b = round((q71.feed*1000*0.1  * q71.ratio_kcalkg).sum(), 0)

# no. of potentially fed humans with the 10% cereals feed
q71c = round(q71b/daily_calorie_req, 0)

q71d = pd.DataFrame(np.array([[q71a, q71b, q71c]]), columns=['10%_feed_kg', '10%_feed_kcal', 'potential_humans_fed'])
q71d

,10%_feed_kg,10%_feed_kcal,potential_humans_fed
0,14042400.0,3.915597e+10,15662387.0


**8. In Thailand, what proportion of cassava is exported? What is the proportion of undernutrition?**

In [48]:
q8 = data2[data2.country == 'Thailand']
q8 = q8[q8.item == 'Cassava and products']
#q8a = q8.item.unique()
q8 = q8[['year','country','item','domestic_supply_quantity', 'export_quantity', 'feed', 'food', 'import_quantity', 'losses', 
         'other_uses', 'processing', 'production', 'seed', 'stock_variation', 'u_percentage']]
q8['proportion_exported_quantity'] = 100*(q8['feed'] + q8['food'] + q8['losses'] + q8['other_uses'])/q8['export_quantity']
q8[['year','country','item','proportion_exported_quantity', 'u_percentage']].set_index('year')

,country,item,proportion_exported_quantity,u_percentage
year,,,,
2014,Thailand,Cassava and products,25.425589,8.036383
2017,Thailand,Cassava and products,28.882315,7.802362
